### Train sentence2vec model

In [1]:
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))

if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

# from pyml4.common import context, db
# context.local_context.print_summary()

import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import logging
import nltk
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
import pymysql.cursors

In [2]:
logger = logging.getLogger(__name__)
sent_detector = None

# store NLTK resources inside ML4
#nltk_download_dir = os.path.dirname(nltk.__file__)
#nltk.data.path.append(os.path.dirname(nltk.__file__))
#nltk.download('punkt', download_dir=os.path.dirname(nltk.__file__))
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

NUM_REVIEWS_TO_FETCH = 1000000

[nltk_data] Downloading package punkt to /home/evazhong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:

class ReviewSentences(object):
    # def __init__(self):
    #     self.db_client = db.manually_connect()
    #
    # def __del__(self):
    #     self.db_client.close()

    def __iter__(self):
        connection = pymysql.connect(host="127.0.0.1",
                                    user="web",
                                    password="atth1132",
                                    db="ML3_mirror")

        with connection.cursor() as cur:
        #with self.db_client.cursor() as cur:
        #cur.execute("select imdbReviewId, movieId, body, upVotes, downVotes from imdb_review ")
            cur.execute("""select imdbReviewId, movieId, body, upVotes, downVotes
                            FROM (select imdbReviewId, movieId, body, upVotes, downVotes,
                            @rn:= if(@movieId = movieId, @rn+1, 1) as rn,
                            @movieId:=movieId as pi
                            from imdb_review, (select @rn:=0) b
                            order by movieId, (upVotes-downVotes)desc) x
                            where x.rn <= 80 and x.pi < 5500
                            """)
            for (review_id, movie_id, body, up, down) in cur:
                for sentence in sent_detector.tokenize(body):
                    yield (movie_id, sentence)

In [5]:
review_sentences = [(movie_id,sentence) for (movie_id,sentence) in ReviewSentences()]

In [6]:
#Remove punctuations
#nltk.download('stopwords')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')

stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/evazhong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'weren', 'some', 'of', "didn't", 'further', 's', 'hadn', 're', 'having', 'until', "don't", 'now', 'myself', 'or', 'up', 'how', 'be', "won't", "you're", 'once', 'himself', 'mightn', 'wouldn', 'under', "wasn't", 'here', 'hasn', 't', 'are', 'doing', 'he', 'just', 'mustn', 'hers', 'am', 'below', 'did', 'only', 'shouldn', 'as', 'the', 'ain', "doesn't", 'both', 'y', 'who', 'by', 'about', 'out', 'wasn', 'themselves', 'off', 'them', 'which', 'more', 'its', 'why', 'so', 'with', 'to', 'whom', "you'd", 'where', "isn't", 'i', 'should', 'than', 'then', 'haven', 'we', 'she', 'each', 'over', 'ourselves', 've', 'that', 'in', 'down', 'and', 'there', 'didn', "weren't", 'they', 'but', 'ma', 'is', 'yourself', 'can', 'an', 'before', 'most', 'again', "haven't", 'for', 'it', 'while', 'no', 'between', 'isn', 'such', 'do', 'd', 'those', "hadn't", "mightn't", 'after', 

In [7]:
stemmer = SnowballStemmer("english")
lemma = WordNetLemmatizer()

# stem and remove stopwords from reviews
review_sent = []
only_sentence = []
review_word = []
#
nltk.download('wordnet')

for i in review_sentences:
    #print("Hi")
    #print(i)
    word_lst = [lemma.lemmatize(word,pos='v') for word in tokenizer.tokenize(i[1].lower()) if (word not in stop_words)]
    review_word.append((i[0],word_lst))
    sent = ' '.join(word_lst)
    review_sent.append((i[0],sent))
    only_sentence.append(sent)
print('finish lemmatizing')    

[nltk_data] Downloading package wordnet to /home/evazhong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
finish lemmatizing


In [11]:
giant_sentence = ' '.join(only_sentence)
tokenized_giant_sentence = word_tokenize(giant_sentence)
tokenized_giant_sentence[0]


# In[252]:


# Bigram words
# filter_stops = lambda w: len(w) < 3
bigram_measures = nltk.collocations.BigramAssocMeasures()
bcf2 = BigramCollocationFinder.from_words(tokenized_giant_sentence)
bcf3 = BigramCollocationFinder.from_words(tokenized_giant_sentence)
bcf4 = BigramCollocationFinder.from_words(tokenized_giant_sentence)
bcf5 = BigramCollocationFinder.from_words(tokenized_giant_sentence)
bcf7 = BigramCollocationFinder.from_words(tokenized_giant_sentence)
bcf9 = BigramCollocationFinder.from_words(tokenized_giant_sentence)
bcf11 = BigramCollocationFinder.from_words(tokenized_giant_sentence)

# bcf.apply_word_filter(filter_stops)
bcf2.apply_freq_filter(2)
bcf3.apply_freq_filter(3)
bcf4.apply_freq_filter(4)
bcf5.apply_freq_filter(5)
bcf7.apply_freq_filter(7)
bcf9.apply_freq_filter(9)
bcf11.apply_freq_filter(11)

bigrams2 = bcf2.nbest(bigram_measures.pmi, 800)
bigrams3 = bcf3.nbest(bigram_measures.pmi, 800)
bigrams4 = bcf4.nbest(bigram_measures.pmi, 800)
bigrams5 = bcf5.nbest(bigram_measures.pmi, 800)
bigrams7 = bcf7.nbest(bigram_measures.pmi, 800)
bigrams9 = bcf9.nbest(bigram_measures.pmi, 400)
bigrams11 = bcf11.nbest(bigram_measures.pmi, 200)

bigrams = bigrams2 + bigrams3+ bigrams4 + bigrams5 + bigrams7+ bigrams9+ bigrams11


print('bigrams')


# make bigrams words into a single word
bigrams_lst = [' '.join(n).lower() for n in bigrams]
bigrams_lst = [n.encode("utf-8") for n in bigrams_lst]
bigrams_lst1 = ['-'.join(n).lower() for n in bigrams]
bigrams_lst1 = [n.encode("utf-8") for n in bigrams_lst1]
bigrams_lst1


# In[ ]:


# Replace bigram words with single words
#sentences = [n.encode('ascii','ignore').lower() for n in sentences]
bigrams_dict = dict(zip(bigrams_lst, bigrams_lst1))


KeyboardInterrupt: 

In [12]:
taggeddoc = [TaggedDocument(words=word_tokenize(sent),tags=[str(movie_id)]) for (movie_id, sent) in review_sent]

In [ ]:
taggeddoc

In [8]:
#sentence_list = []
#for movieId, sentence in review_word:
    #sentence_list.append(sentence)


In [25]:
#sentence_list[1]

['always',
 'look',
 'newest',
 'technological',
 'possibilities',
 'use',
 'movies',
 'create',
 'movies',
 'worth',
 'watch',
 'even',
 'make',
 'decade',
 'ago']

In [ ]:
print('start training model3')
doc_vec_model2 = gensim.models.Doc2Vec(taggeddoc, alpha=0.025, size=1000, min_alpha=0.025,min_count=10,workers=35,iter=20)
#word_vec_model1.train(only_sentence,total_examples=word_vec_model1.corpus_count,epochs=word_vec_model1.iter)
doc_vec_model2.save('Doc2Vec_fullds_model2')
print('doc2vec model trained')

start training model3


/home/evazhong/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/evazhong/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


### Load word2vec model 

In [1]:
import gensim
from gensim.models import Word2Vec

In [2]:
word_vec_model2 = Word2Vec.load('Word2Vec_fullds_model2')

In [3]:
word_vec_model2.wv.vocab

{'big': <gensim.models.keyedvectors.Vocab at 0x7f38faabf5f8>,
 'fan': <gensim.models.keyedvectors.Vocab at 0x7f38faabf6d8>,
 'animate': <gensim.models.keyedvectors.Vocab at 0x7f38faabf748>,
 'movies': <gensim.models.keyedvectors.Vocab at 0x7f38faabf7b8>,
 'come': <gensim.models.keyedvectors.Vocab at 0x7f38faabf828>,
 'pixar': <gensim.models.keyedvectors.Vocab at 0x7f38faabf898>,
 'studios': <gensim.models.keyedvectors.Vocab at 0x7f38faabf908>,
 'always': <gensim.models.keyedvectors.Vocab at 0x7f38faabf978>,
 'look': <gensim.models.keyedvectors.Vocab at 0x7f38faabf9e8>,
 'newest': <gensim.models.keyedvectors.Vocab at 0x7f38faabfa58>,
 'technological': <gensim.models.keyedvectors.Vocab at 0x7f38faabfa90>,
 'possibilities': <gensim.models.keyedvectors.Vocab at 0x7f38faabfac8>,
 'use': <gensim.models.keyedvectors.Vocab at 0x7f38faabfb38>,
 'create': <gensim.models.keyedvectors.Vocab at 0x7f38faabfba8>,
 'worth': <gensim.models.keyedvectors.Vocab at 0x7f38faabfc18>,
 'watch': <gensim.models

In [4]:
similar_terms = {}

In [5]:
for index, row in unique_tags.iterrows():
    tag = row['tags']
    if tag in word_vec_model2.wv.vocab:
        similar_terms[tag] = word_vec_model2.wv.most_similar(tag)

NameError: name 'unique_tags' is not defined

In [3]:
word_vec_model2 = Word2Vec.load('Word2Vec_fullds_model2')

In [5]:
len(word_vec_model2.wv.vocab)

65012

In [4]:
word_vec_model2.wv.similar_by_word("happy")

[('happier', 0.27740973234176636),
 ('happily', 0.2672647535800934),
 ('happiest', 0.22986197471618652),
 ('upbeat', 0.22156524658203125),
 ('glad', 0.20897459983825684),
 ('optimistic', 0.20554432272911072),
 ('hopeful', 0.20425133407115936),
 ('unhappy', 0.19973137974739075),
 ('cheerful', 0.19736379384994507),
 ('awww', 0.18560005724430084)]

### Sentence2Vec by averaging word2vec

1st round:
- for each tag, calculate embedding
- if a phrase:
    - split into words;
    - if both in model, embedding is average;
    - store phrase embedding

2nd round:
- for each word, iterate database and find the most similar word/phrases

